# Simulations in `vipdopt`

This notebook serves as an interactive guide to creating simulations in `vipdopt` to interface directly with the Lumerical FDTD solver. First run the below code to do the necessary setup and imports.

In [1]:
# imports
from pathlib import Path
import sys  

import numpy as np

np.set_printoptions(threshold=100)

# Get vipdopt directory path from Notebook
parent_dir = str(Path().resolve().parents[2])

# Add to sys.path
sys.path.insert(0, parent_dir)

# Imports from vipdopt
from vipdopt.simulation import LumericalSimulation, LumericalSimObject, Monitor, Source, Import, LumericalFDTD

## Creating a New Simulation

To create a simulation we must first instantiate a `LumericalSimulation` object. You can also load a simulation from an appropiately formatted JSON file.

In [ ]:
sim = LumericalSimulation()

loaded_sim = LumericalSimulation(source='simulation_example.json')

## Simulation Objects

Once a simulation object has been created, it can be populated with objects. Simulation objects require three things:

1. A name
2. An `LumericalSimObjectType`, that is, the type of object to create in Lumerical (e.g. power, profile, dipole)
3. Properties, the values that define the object

The properties can be provided upon creation of the object, or manipulated after the fact.

In [ ]:
# Here we create an object through our simulation
props = {
    'x' : 0.0,
    'y' : 0.4e-6,
    'monitor type' : 'linear x',
}

power = sim.new_object('power', 'power', properties=props)

# You can also create a standalone object and add it to your simulation

source_1 = LumericalSimObject('source_1', 'dipole') 
sim.add_object(source_1)

print(sim.objects)

Simulation objects can be edited with the update method, which works just like that of the builtin `dict` in Python. Simulation objects can also be edited using dictionary-like access to individual properties.

In [ ]:
# Using the update method
source_props = {
    'theta': 90,
}

source_1.update(x=0, y=0)
source_1.update(**source_props)  # You can also unpack a dictionary

# The simulation's update_object method works in a similar fashion
sim.update_object('source_1', phi=0)

source_1

In [ ]:
# Using dictionary access to change parameters and add new ones
source_1['theta'] = 45
source_1['z'] = 0

source_1

## Simulation Object Subclasses

Because the different object types have unique properties and data from simulations, there are also subclasses of `LumericalSimObject` that better cater to their specific needs. These include:

* `Monitor` (which is further split into the `Profile` and `Power` classes)
* `Source` (which is further split into `DipoleSource`, `GaussianSource`, and `TFSFSource`)
* `Import`

### `Monitor` Class

The `Monitor` class has additional methods and attributes for accessing data from a previously ran simulation. In Lumerical, profile and power monitors measure specific data: 

* E field
* H field
* Poynting vector
* transmission
* power
* source power

When a `Monitor` is first created, these values are all set to `None`.

In [ ]:
mon = Monitor('mon', 'power')

print(mon.e)  # will be None by default

Once a simulation is run, files containing each monitor's data can be generated. `Monitor` objects will dynamically draw data from these files as needed, and repalce its data with placeholder values (`None`) when no longer needed, so as to save memory.

In [ ]:
# Suppose simulation 'sim' was run and saved data to 'sim.fsp'
# The data from mon1 would be saved to 'sim_mon1.npz'

mon.set_source('sim_mon.npz')  # Link our monitor to it's data file

print(mon.e)  # No longer None

# Say we wanted the average absolute value of the electric field
value = np.abs(mon.e).mean()

# We no longer need the field in memory so we can clear it out; `value` remains the same
mon.reset()

print(f'The mean value of |E| is: {value}')

### `Source` Class

The `Source` class is made to represent the various light sources in a simulation. The main addition to the `Source` class is that it is hashable, so it can be used as a key in dictionaries.

The `Source` class is currently required for creating figures of merit (see more [here](fom.ipynb#using-the-fom-class))

In [ ]:
src1 = Source('src1', 'gaussian')
src2 = Source('src2', 'dipole')

# A dictionary that maps Sources to their type
source_to_type = {src: src.obj_type for src in [src1, src2]}
print(source_to_type)

### `Import` Class

The `Import` class mirrors the import primitive in Lumerical. It is used to create a 3D geometry. If you wish to use `vipdopt` for optimization purposes, you will need to include at least one `Import` in your simulation, as it is responsible for creating the design space for a device.

`Import` comes with two additional methods, `set_nk2` and `get_nk2` which mirrors Lumerical's import_nk2 script command. These just set and retrieve refractive index values (n and k) over a volume (defined by x, y, z). These methods are mainly used by the `LumericalOptimization` to step a design and import those changes into Lumerical, and can mainly be ignored.

## Connecting to Lumerical FDTD

So far, all we've done is create a Python representation of a simulation, but to actually use it with Lumerical, we would typically use their provided API (`lumapi`). In `vipdopt`, a class `LumericalFDTD` is provided which effectively serves as a wrapper around `lumapi` functionality. It is responsible for the following:

* Importing `LumericalSimulation`'s into Lumerical
* Saving and running simulation jobs
* Retrieving data from completed simulations
* Managing resources to be used in running jobs

In [6]:
# Creating the FDTD hook

fdtd = LumericalFDTD()
sim = LumericalSimulation('simulation_example.json')
sim_file = 'sim.fsp'  # Where Lumerical will save simulation data

fdtd.connect()  # This starts a Lumerical session

fdtd.load(path=None, sim=sim)  # Load simulation into Lumerical
fdtd.save(sim_file)  # Lumerical must have a file on the disk before running a job

There are two ways to run a simulation:

1. Load into the fdtd and run (only runs the currently loaded simulation)
2. Add it as a job `fdtd.addjob()` and use `fdtd.runjobs()`

In [4]:
# Option 1
fdtd.run()

In [7]:
# Option 2

fdtd.addjob(sim_file)  # Add the simulation file to the job queue
fdtd.runjobs()

In [8]:
# Create individual data files for each Monitor
fdtd.reformat_monitor_data([sim])

fdtd.close()  # End Lumerical session